This notebook is based on [an official 🤗 notebook - "How to fine-tune a model on text classification"](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb). The main aim of this notebook is to show the process of conversion from vanilla 🤗 to Ray AIR 🤗 without changing the training logic unless necessary.

If you do not have 🤗 Datasets and 🤗 Transformers installed locally, uncomment and run the following line:

In [ ]:
#! pip install datasets transformers

We will use a [runtime enviroment](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments) to ensure that we have 🤗 Datasets and 🤗 Transformers installed on the Ray cluster.

In [1]:
import ray

runtime_env = {
    "pip": [
        "torch==1.10.0",  # required to pass
        "datasets",
        "git+https://github.com/huggingface/transformers"  # use master version due to a bug in 4.18 causing an exception during training
    ],
}
ray.init(address="auto", runtime_env=runtime_env)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.5', ray_version='2.0.0.dev0', ray_commit='d4a906e177b6738627307737bce92730b00de323', address_info={'node_ip_address': '172.31.65.134', 'raylet_ip_address': '172.31.65.134', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-02_12-01-48_903878_176/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-02_12-01-48_903878_176/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-02_12-01-48_903878_176', 'metrics_export_port': 62024, 'gcs_address': '172.31.65.134:9031', 'address': '172.31.65.134:9031', 'node_id': '5c7a90d6adab9b54c21f70016ea7350b2b36ce00a060d58202b46bb1'})

Make sure your version of Transformers on the cluster is at least 4.19.0:

In [2]:
@ray.remote
def print_transformers_version():
    import transformers
    print(transformers.__version__)

ray.get(print_transformers_version.remote())

(print_transformers_version pid=22713) 4.19.0.dev0


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will be running the training on a Ray Cluster using Ray AIR.

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [4]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions.

As Ray AIR doesn't provide integrations for 🤗 Datasets yet, we will simply run the normal 🤗 Datasets code in a Ray Task, so that execution happens on the Ray cluster. `load_metric_fn` will be used inside the `HuggingFaceTrainer` later. Note that we are only defining the functions here, and not running them. We will use them later.

In [5]:
from datasets import load_dataset, load_metric
actual_task = "mnli" if task == "mnli-mm" else task

def load_dataset_fn():
    return load_dataset("glue", actual_task)

def load_metric_fn():
    return load_metric('glue', actual_task)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric).

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

We continiue with our pattern of defining functions and variables which we will use in the final Ray Task.

In [6]:
from transformers import AutoTokenizer

def load_tokenizer_fn() -> AutoTokenizer:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    return tokenizer

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [7]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [9]:
def preprocess_function(examples, *, tokenizer):
    sentence1_key, sentence2_key = task_to_keys[task]
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [10]:
def encode_dataset(dataset, tokenizer):
    return dataset.map(preprocess_function, batched=True, fn_kwargs=dict(tokenizer=tokenizer))

For Ray AIR, instead of using 🤗 Dataset objects directly, we will convert them to Ray Datasets. As both are backed by Arrow tables, the conversion is quite simple.

In [11]:
import ray.data
from datasets import DatasetDict

def convert_hf_dataset_to_ray_dataset(hf_dataset):
    if isinstance(hf_dataset, DatasetDict):
        return {k: ray.data.from_arrow(v.data.table) for k, v in hf_dataset.items()}
    return ray.data.from_arrow(hf_dataset.data.table)

Finally, we will tie it all up in one Ray Task. It will return a dictionary of Ray Datasets:

In [12]:
@ray.remote
def load_and_preprocess_dataset():
    dataset = load_dataset_fn()
    tokenizer = load_tokenizer_fn()
    encoded_dataset = encode_dataset(dataset, tokenizer)
    return convert_hf_dataset_to_ray_dataset(encoded_dataset)

ray_datasets = ray.get(load_and_preprocess_dataset.remote())

(load_and_preprocess_dataset pid=22713) Reusing dataset glue (/home/ray/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 986.66it/s]3) 
(load_and_preprocess_dataset pid=22713) Loading cached processed dataset at /home/ray/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f14dcaf95fc91eb3.arrow
100%|██████████| 2/2 [00:00<00:00, 70.71ba/s]13) 
(load_and_preprocess_dataset pid=22713) Loading cached processed dataset at /home/ray/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-050a2ed8debaa978.arrow


## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class.

We will not go into details about each specific component of the training (see the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb) for that).

The main difference when using the Ray AIR  is that we need to create our 🤗 Transformers `Trainer` inside a function (`trainer_init_per_worker`) and return it. That function will be passed to the `HuggingFaceTrainer` and ran on every Ray worker. The training will then proceed by the means of PyTorch DDP.

Make sure that you initialize the model, metric and tokenizer inside that function. Otherwise, you may run into serialization errors.

Please note that if you don't want to use CUDA, you need to explicitly set `no_cuda=True` inside the `TrainingArguments`. Furthermore, `push_to_hub=True` is not yet supported. Ray will however checkpoint the model at every epoch, allowing you to push it to hub manually. We will do that after the training.

If you wish to use thrid party logging libraries, such as MLFlow or Weights&Biases, do not set them in `TrainingArguments` (they will be automatically disabled) - instead, you should be passing Ray AIR callbacks to `HuggingFaceTrainer`'s `run_config`.

We also set `disable_tqdm=True` to declutter the output a little.

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    metric = load_metric_fn()
    tokenizer = load_tokenizer_fn()
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    args = TrainingArguments(
        f"{model_name}-finetuned-{task}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=2,
        weight_decay=0.01,
        push_to_hub=False,
        disable_tqdm=True,
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print("Starting training")
    return trainer

With our `trainer_init_per_worker` complete, we can now instantiate the `HuggingFaceTrainer`. Aside from the function, we set the `scaling_config`, controlling the amount of workers and resources used, and the `datasets` we will use for training and evaluation.

We will use 2 workers, each with a single GPU assigned.

In [23]:
from ray.ml.train.integrations.huggingface import HuggingFaceTrainer

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config={"num_workers": 2, "use_gpu": True},
    datasets={"train": ray_datasets["train"], "evaluation": ray_datasets[validation_key]},
)

Finally, we call the `fit` method to being training with Ray AIR. We will save the `Result` object to a variable so we can access metrics and checkpoints.

In [32]:
result = trainer.fit()

(pid=2247, ip=172.31.74.151) comet_ml is installed but `COMET_API_KEY` is not set.


Trial name,status,loc
HuggingFaceTrainer_edeb1_00000,RUNNING,172.31.74.151:2247


(BaseWorkerMixin pid=2284, ip=172.31.74.151) 2022-05-02 13:46:31,885	INFO torch.py:346 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=2477, ip=172.31.89.172) 2022-05-02 13:46:31,906	INFO torch.py:346 -- Setting up process group for: env:// [rank=1, world_size=2]
(BaseWorkerMixin pid=2284, ip=172.31.74.151) comet_ml is installed but `COMET_API_KEY` is not set.
(BaseWorkerMixin pid=2477, ip=172.31.89.172) comet_ml is installed but `COMET_API_KEY` is not set.


Trial name,status,loc
HuggingFaceTrainer_edeb1_00000,RUNNING,172.31.74.151:2247


(BaseWorkerMixin pid=2284, ip=172.31.74.151) Is CUDA available: True
(BaseWorkerMixin pid=2477, ip=172.31.89.172) Is CUDA available: True


...

2022-05-02 13:52:31,173	INFO command_runner.py:394 -- Fetched IP: 172.31.74.151
2022-05-02 13:52:31,173	INFO log_timer.py:25 -- NodeUpdater: ins_MK4KKJz1fawcXn8Xt79sfRVp: Got IP  [LogTimer=42ms]


2022-05-02 13:52:31,575	WARNING util.py:214 -- The `process_trial_save` operation took 2.087 s, which may be a performance bottleneck.


Result for HuggingFaceTrainer_edeb1_00000:
  _time_this_iter_s: 55.85324740409851
  _timestamp: 1651524600
  _training_iteration: 3
  date: 2022-05-02_13-50-01
  done: true
  epoch: 2.0
  eval_loss: 0.48282191157341003
  eval_matthews_correlation: 0.4595994050968516
  eval_runtime: 1.0754
  eval_samples_per_second: 490.991
  eval_steps_per_second: 15.808
  experiment_id: 8b873c2e53394b578e82dc3a6e3feb6c
  experiment_tag: '0'
  hostname: ip-172-31-74-151
  iterations_since_restore: 3
  node_ip: 172.31.74.151
  pid: 2247
  should_checkpoint: true
  step: 536
  time_since_restore: 212.14981865882874
  time_this_iter_s: 14.975391387939453
  time_total_s: 212.14981865882874
  timestamp: 1651524601
  timesteps_since_restore: 0
  train_loss: 0.47598597896632866
  train_runtime: 196.3277
  train_samples_per_second: 43.682
  train_steps_per_second: 2.73
  training_iteration: 3
  trial_id: edeb1_00000
  warmup_time: 0.004483461380004883
  


Trial name,status,loc,iter,total time (s),eval_loss,eval_matthews_correlation,eval_runtime
HuggingFaceTrainer_edeb1_00000,TERMINATED,172.31.74.151:2247,3,212.15,0.482822,0.459599,1.0754


2022-05-02 13:52:31,694	INFO tune.py:747 -- Total run time: 367.78 seconds (367.62 seconds for the tuning loop).


You can use the returned `Result` object to access metrics and the Ray AIR `Checkpoint` associated with the last iteration.

In [33]:
result

Result(metrics={'eval_loss': 0.48282191157341003, 'eval_matthews_correlation': 0.4595994050968516, 'eval_runtime': 1.0754, 'eval_samples_per_second': 490.991, 'eval_steps_per_second': 15.808, 'epoch': 2.0, 'step': 536, 'train_runtime': 196.3277, 'train_samples_per_second': 43.682, 'train_steps_per_second': 2.73, 'train_loss': 0.47598597896632866, '_timestamp': 1651524600, '_time_this_iter_s': 55.85324740409851, '_training_iteration': 3, 'time_this_iter_s': 14.975391387939453, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 3, 'trial_id': 'edeb1_00000', 'experiment_id': '8b873c2e53394b578e82dc3a6e3feb6c', 'date': '2022-05-02_13-50-01', 'timestamp': 1651524601, 'time_total_s': 212.14981865882874, 'pid': 2247, 'hostname': 'ip-172-31-74-151', 'node_ip': '172.31.74.151', 'config': {}, 'time_since_restore': 212.14981865882874, 'timesteps_since_restore': 0, 'iterations_since_restore': 3, 'warmup_time': 0.004483461380004883, 'expe

You can now use the checkpoint to run prediction with `HuggingFacePredictor`, which wraps around [🤗 Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines). In order to distribute prediction, we use `BatchPredictor`. While this is not necessary for the very small example we are using (you could use `HuggingFacePredictor` directly), it will scale well to a large dataset.

In [35]:
from ray.ml.predictors.integrations.huggingface import HuggingFacePredictor
from ray.ml.batch_predictor import BatchPredictor
import pandas as pd

sentences = ['Bill whistled past the house.',
  'The car honked its way down the road.',
  'Bill pushed Harry off the sofa.',
  'the kittens yawned awake and played.',
  'I demand that the more John eats, the more he pay.']
predictor = BatchPredictor.from_checkpoint(
    result.checkpoint,
    HuggingFacePredictor,
    task="text-classification",
)
data = ray.data.from_pandas(pd.DataFrame(sentences, columns=["sentence"]))
prediction = predictor.predict(data)
prediction = prediction.to_pandas()
prediction

Map Progress (2 actors 1 pending):   0%|          | 0/1 [00:08<?, ?it/s](BlockWorker pid=26751) 2022-05-02 13:53:31.081365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(BlockWorker pid=26751) 2022-05-02 13:53:31.081392: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
(BlockWorker pid=26751) 2022-05-02 13:53:31.081411: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-65-134): /proc/driver/nvidia/version does not exist
(BlockWorker pid=26751) 2022-05-02 13:53:31.081674: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performan

,label,score
0,LABEL_1,0.959990
1,LABEL_1,0.961284
2,LABEL_1,0.960226
3,LABEL_1,0.957515
4,LABEL_1,0.936557


To be able to share your model with the community, there are a few more steps to follow.

We have conducted the training on the Ray cluster, but share the model from the local enviroment - this will allow us to easily authenticate.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Now, load the model and tokenizer locally, and recreate the `Trainer`:

In [38]:
hf_trainer = HuggingFaceTrainer.load_huggingface_checkpoint(result.checkpoint, AutoModelForSequenceClassification, AutoTokenizer)

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
hf_trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```